Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

L2正则化

In [24]:
def forward_propagation(features, weights, biases):
  hidden_layer = tf.add(tf.matmul(features, weights[0]), biases[0])
  hidden_layer = tf.nn.relu(hidden_layer)    
  logits = tf.matmul(hidden_layer, weights[1]) + biases[1]
  return logits


batch_size = 128
hidden_nodes_num = 1024
reglarziation_rate = 0.01
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.

  hidden_layer_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_num]))
  hidden_layer_biases = tf.Variable(tf.zeros([hidden_nodes_num]))

  out_weights = tf.Variable(
    tf.truncated_normal([hidden_nodes_num, num_labels]))
  out_biases = tf.Variable(tf.zeros([num_labels]))

  # Weights and biases
  weights = [hidden_layer_weights, out_weights]
  biases = [hidden_layer_biases, out_biases]
  
  # Training computation.
  logits = forward_propagation(tf_train_dataset, weights, biases)
  # key code: L2 regularization
  l2_loss = tf.nn.l2_loss(weights[0]) + tf.nn.l2_loss(weights[1])  
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
    ) + reglarziation_rate * l2_loss
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_propagation(tf_valid_dataset, weights, biases))
  test_prediction = tf.nn.softmax(forward_propagation(tf_test_dataset, weights, biases))

In [25]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3435.180420
Minibatch accuracy: 7.0%
Validation accuracy: 33.9%
Minibatch loss at step 500: 21.275702
Minibatch accuracy: 85.2%
Validation accuracy: 84.4%
Minibatch loss at step 1000: 0.796504
Minibatch accuracy: 85.9%
Validation accuracy: 83.6%
Minibatch loss at step 1500: 0.799122
Minibatch accuracy: 81.2%
Validation accuracy: 83.6%
Minibatch loss at step 2000: 0.726605
Minibatch accuracy: 86.7%
Validation accuracy: 83.5%
Minibatch loss at step 2500: 0.597418
Minibatch accuracy: 87.5%
Validation accuracy: 83.5%
Minibatch loss at step 3000: 0.693392
Minibatch accuracy: 87.5%
Validation accuracy: 83.3%
Test accuracy: 90.0%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

使用很少batches训练，会使得模型在training data上准确度非常高，甚至达到100%，但是在valid data和test data表现不会太好

In [28]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    # Restrict your training data to just a few batches. 
    tmp_step = step % 5
    offset = (tmp_step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3535.855469
Minibatch accuracy: 3.9%
Validation accuracy: 35.8%
Minibatch loss at step 500: 21.140062
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 1000: 0.534120
Minibatch accuracy: 100.0%
Validation accuracy: 79.4%
Minibatch loss at step 1500: 0.367188
Minibatch accuracy: 100.0%
Validation accuracy: 79.4%
Minibatch loss at step 2000: 0.354245
Minibatch accuracy: 100.0%
Validation accuracy: 79.3%
Minibatch loss at step 2500: 0.347761
Minibatch accuracy: 100.0%
Validation accuracy: 79.5%
Minibatch loss at step 3000: 0.343241
Minibatch accuracy: 100.0%
Validation accuracy: 79.5%
Test accuracy: 86.2%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [35]:
def forward_propagation(features, weights, biases):
  hidden_layer = tf.add(tf.matmul(features, weights[0]), biases[0])
  hidden_layer = tf.nn.relu(hidden_layer)    
  logits = tf.matmul(hidden_layer, weights[1]) + biases[1]
  return logits


def forward_propagation_with_dropout(features, weights, biases):
  hidden_layer = tf.add(tf.matmul(features, weights[0]), biases[0])
  hidden_layer = tf.nn.dropout(tf.nn.relu(hidden_layer), keep_prob=keep_prob)    
  logits = tf.matmul(hidden_layer, weights[1]) + biases[1]
  return logits

batch_size = 128
hidden_nodes_num = 1024
keep_prob = 0.4
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that wibll be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.

  hidden_layer_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_num]))
  hidden_layer_biases = tf.Variable(tf.zeros([hidden_nodes_num]))

  out_weights = tf.Variable(
    tf.truncated_normal([hidden_nodes_num, num_labels]))
  out_biases = tf.Variable(tf.zeros([num_labels]))

  # Weights and biases
  weights = [hidden_layer_weights, out_weights]
  biases = [hidden_layer_biases, out_biases]
  
  # Training computation.
  logits = forward_propagation_with_dropout(tf_train_dataset, weights, biases)
  # key code: L2 regularization
  l2_loss = tf.nn.l2_loss(weights[0]) + tf.nn.l2_loss(weights[1])  
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
    ) + reglarziation_rate * l2_loss
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_propagation(tf_valid_dataset, weights, biases))
  test_prediction = tf.nn.softmax(forward_propagation(tf_test_dataset, weights, biases))

In [36]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    # Restrict your training data to just a few batches. 
    # tmp_step = step % 5
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3606.531006
Minibatch accuracy: 10.2%
Validation accuracy: 40.2%
Minibatch loss at step 500: 21.554596
Minibatch accuracy: 81.2%
Validation accuracy: 83.7%
Minibatch loss at step 1000: 0.920880
Minibatch accuracy: 85.2%
Validation accuracy: 83.3%
Minibatch loss at step 1500: 0.939736
Minibatch accuracy: 78.9%
Validation accuracy: 83.2%
Minibatch loss at step 2000: 0.807197
Minibatch accuracy: 83.6%
Validation accuracy: 83.2%
Minibatch loss at step 2500: 0.749921
Minibatch accuracy: 85.2%
Validation accuracy: 83.0%
Minibatch loss at step 3000: 0.785033
Minibatch accuracy: 85.2%
Validation accuracy: 82.8%
Test accuracy: 89.4%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


### Network Architecture 

input layer: 28 * 28
hidden layers: 1024-300-50
output layer: 10
learning rate: linear decay of the learning rate or tf.train.exponential_decay
dropout keep prob: 0.5  